# Week 4

During week 4 of the project, we worked on adding the sound effects. In order to do so, we first used MATLAB to convert some `wav` audio files into a large array and then used that array to send data directly to the DAC using DMA.

Next, we tweaked the background color a bit to make the display more presentable.

---

## The Design

#### 1. Converting audio files to arrays

In order to add the audio in game, we first had to covert it into a form which can directly be read by the microcontroller. Moreover, it had to be in a format that can be directly pushed to the DAC. Hence, we also had to append the 4 DAC configuration bits in the MSB of each element of the audio array. To do so, we used a MATLAB script which can be found [here](https://parthssharma.github.io/ECE4760/FinalProject/Files/ConvertAudioToArray.txt).

First, we defined the upper and lower bounds of the audio file in terms of the audio sample number. This was done to remove any unwanted sections of the audio. Next, we defined the `FILE_NAME` and the `TEXT_FILE`. These are the audio input file name and the output text file name respectively. Lastly, we defined the `BitRate` which is the new sampling frequency of the audio. This was done to reduce the size of the audio array. _Note: Reducing the bitrate of the audio also reduces the audio quality._

```c
UPPER_RANGE = %Your Upper Range%
LOWER_RANGE = %Your Upper Range%
FILE_NAME = %Audio File Name%
TEXT_FILE = %Text File Name%
BitRate = 8000
Range = [LOWER_RANGE, UPPER_RANGE];
```

<br>

Next we used the `audioread()` function to read the audio file into an array `y` and get its default sampling frequency `Fs`. In order to change the sampling frequency, we used the `resample()` function to change its bitrate. _Note: This function requires the DSP toolbox to be installed in MATLAB to work properly._ The audio file we read was a 2 channel audio, hence we sliced it down to a single channel and stored it in `ys` as a column matrix. The amplitude of the audio input stored in `ys` ranges from -1 to 1. However, we have a 12-bit DAC. Therefore, we had to rescale the amplitude from 0 to 4095. This was done using the `rescale()` function. Laslty, we force casted each element of the audio to a `uint16` data type in order to make it compatible with the DAC.

```c
[y, Fs] = audioread(FILE_NAME, Range);
yn = resample(y, BitRate, Fs);
ys = yn(:, 1);
yqfinal = rescale(ys, 0, 4095);
ysfinal = cast(yqfinal, 'uint16');
```

<br>

We then opened the text file in write mode using the `fopen()` function. Then we used a `for` loop to traverse through every element of the audio array and append the DAC configuration bits to it by performing a bitwise `OR` operation using the `bitor()` function. The elements are then written into the text file using the `fprintf()` function. After all elements are written in the text file, we closed the file using the `fclose()` function.

```c
file = fopen(TEXT_FILE, 'w');
for i = 1 : length(ysfinal)
    yfinal(i) = bitor(ysfinal(i), 0b0011000000000000);
    fprintf(file, "%d, ", yfinal(i));
end
fclose(file);
```
<br>

#### 2. Creating the audio header file

The above conversion was repeated for all audio files and we stored the resultant arrays in a file called `Audio.h` which can be found [here](https://parthssharma.github.io/ECE4760/FinalProject/Files/Audio.h). Every single array is of the type `const unsigned short`. We used `const` because the audio will remain constant throught the execution of the program and will be stored in the flash memory.

#### 3. Implementing audio using DMA

In order to implement the audio functionality, we used Direct Memory Access (DMA). We chose DMA because it is a hardware peripheral functionality and as a result doesn't use up any clock cycles to run. Moreover, once setup, it can be triggered from any point in the code. We implemented the code as below.

We first included the audio library.

```c
#include "Audio.h"
```

<br>

We the defined a few parameters. These are namely the number of bytes in the audio arrays (twice the number of elements in the arrays since it is a short data type) and the DMA channels (we are using channels 2 and 3 as channels 0 and 1 are used by the `serial` protothread). We also defined a `SPI_CLK_DIV` as the maximum SPI frequency is 20MHz.

```c
#define JUMP_AUDIO_SIZE 23156 
#define DEAD_AUDIO_SIZE 35670
#define DMA_CHANNEL2 2
#define DMA_CHANNEL3 3
#define SPI_CLK_DIV 2
```

<br>

In order to use the DMA, we need to configure a timer which overflows and throws an interrupt at the same rate as the bitrate of the audio. This is to ensure that the audio is played at the correct frequency. In order to do so, we used the `Timer 3` in PIC32 and configured it to throw an interrupt after every 5000 CPU cycles (since our bitrate is 8kHz and the CPU frequency is 40MHz).

```c
OpenTimer3(T3_ON | T3_SOURCE_INT | T3_PS_1_1, 5000);
```

<br>

Next, we configured the SPI channel 2 to communicate with the DAC. To do so, we enabled the SPI in 16-bit mode and in the master configuration. We also used `SPICON_FRMEN` and `SPICON_FRMPOL` flags to ensure that the SPI is configured in the framed SPI mode as we cannot toggle the chip select line in software while using DMA. Next, we mapped the SPI data out pin to `RPB5` and the SPI chip select line to `RPA3` pin on the board.

```c
SpiChnOpen(SPI_CHANNEL2, SPI_OPEN_ON | SPI_OPEN_MODE16 | SPI_OPEN_MSTEN | SPI_OPEN_CKE_REV | SPICON_FRMEN | SPICON_FRMPOL, SPI_CLK_DIV);
PPSOutput(2, RPB5, SDO2);
PPSOutput(4, RPA3, SS2);
```

<br>

In order to open the DMA channel, we used the `DmaChnOpen()` command. We used the higher priority for `deadAudio` as that takes priority over jumps. Moreover, we used the `DMA_OPEN_DEFAULT` flag as we want our audio samples to play only once and then stop. Next, in order to configure the transfers, we used the `DmaChnSetTxfer()` function to set the transfer parameters. It takes in the arguments as the dma channel, the source address, the destination address, the source size, the destination size and the transferred cell size in the given order. Lastly, we setup the DMA event control to timer 3 IRQ as it will control the DMA transfers once the DMA channel has been enabled.

```c
DmaChnOpen(DMA_CHANNEL2, 3, DMA_OPEN_DEFAULT);
DmaChnSetTxfer(DMA_CHANNEL2, jumpAudio, (void*)&SPI2BUF, JUMP_AUDIO_SIZE, 2, 2);
DmaChnSetEventControl(DMA_CHANNEL2, DMA_EV_START_IRQ(_TIMER_3_IRQ));
    
DmaChnOpen(DMA_CHANNEL3, 0, DMA_OPEN_DEFAULT);
DmaChnSetTxfer(DMA_CHANNEL3, deadAudio, (void*)&SPI2BUF, DEAD_AUDIO_SIZE, 2, 2);
DmaChnSetEventControl(DMA_CHANNEL3, DMA_EV_START_IRQ(_TIMER_3_IRQ));
```

<br>

In order to play the audio, we used the `DmaChnEnable()` function to start DMA transfers to the DAC. This function is called whenever the dino jumps to play the jump audio or the dino dies to play the death audio.

The complete C code for the end of week 4 can be found [here](https://parthssharma.github.io/ECE4760/FinalProject/Files/Week4Code.c). A video demonstration at the end of week 4 is attached below.

<div style="display: flex; justify-content: center;">
    <iframe width="560" height="315" src="https://www.youtube.com/embed/wvb0nBX4MC4" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen>
    </iframe>
</div>
<figure>
    <center><figcaption>Implementation of the audio functionality in the Dino game</figcaption></center>
</figure>

---